# RC Car Notebook for predicting Camera Height

In [1]:
import sys
from matplotlib import pyplot
import os
import azureml
from azureml.core import Experiment
from azureml.core import Environment
from azureml.core import Workspace, Datastore, Dataset
ws = Workspace.from_config()
print("Azure ML SDK Version: ", azureml.core.VERSION)
print(ws.name, ws.location, ws.resource_group, sep='\t')
from azureml.core import Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core.conda_dependencies import CondaDependencies

Azure ML SDK Version:  1.22.0
machinelearninglab	canadacentral	rg_machine_learning


In [2]:
import tensorflow as tf
import keras
from keras.utils import to_categorical
from keras.models import Sequential, load_model
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import MaxPool2D, MaxPooling2D, Dropout, BatchNormalization
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dropout
from keras.optimizers import SGD, Adam
import sqlite3
import yaml
import csv
import numpy
import math
import cv2

Using TensorFlow backend.


In [3]:
from azureml.core import Workspace, Dataset

subscription_id = '103da3c6-1089-4d2b-8b1d-5159e8f5ffc5'
resource_group = 'RG_Machine_Learning'
workspace_name = 'MachineLearningLab'

workspace = Workspace(subscription_id, resource_group, workspace_name)

#datasetImages = Dataset.get_by_name(workspace, name='roadImages4800')
#datasetImages.download(target_path='./images', overwrite=True)

#datasetYaml = Dataset.get_by_name(workspace, name='yamlDataset4800')
#datasetYaml.download(target_path='./yaml', overwrite=True)

#datasetImages = Dataset.get_by_name(workspace, name='100roadImages')
#datasetImages.download(target_path='./test', overwrite=True)


In [4]:
#this takes about 7 minutes to run: loads in all the data from the files - don't run unless you need it
from PIL import Image
img_array = []
dictYaml = []
fileYaml = ''
listOfEverything = []
data_tuple = ()
counter = 0
for fileImg in os.listdir('images'):
    #print(fileImg)
    imageFilePlace = 'images/'+fileImg
    fileYaml = fileImg[:-4]+'.yaml'
    yamlFilePlace = 'yaml/'+fileYaml
    #print(fileYaml)
    with open(os.fsdecode(yamlFilePlace)) as filey:
        dictYaml = yaml.safe_load(filey)
        #print(dictYaml)
    img = Image.open(imageFilePlace)
    img_array = numpy.array(img)
    data_tuple = (img_array, dictYaml)
    listOfEverything.append(data_tuple)
    print(counter)
    counter+=1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [6]:
import random
import pickle
import sklearn
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from cv2 import cvtColor, GaussianBlur
from PIL import Image
print("Tensorflow version " + tf.__version__)
print('Keras Version '+ keras.__version__)
print("Running on GPU:", tf.test.is_built_with_cuda())
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

#DO THIS WHEN YOUR COMPUTE HAS GPU POWER
#gpus= tf.config.experimental.list_physical_devices('GPU')
#tf.config.experimental.set_memory_growth(gpus[0], True)

Tensorflow version 2.1.0
Keras Version 2.3.1
Running on GPU: True
Num GPUs Available:  0


In [7]:
def makeListOfX0(listEverything):
    listX0 = []
    for item in listEverything:
        listX0.append(item[1]['X0'])
    return listX0

In [8]:
def makeListOfYAW0(listEverything):
    listYAW0 = []
    for item in listEverything:
        listYAW0.append(item[1]['YAW0'])
    return listYAW0

In [9]:
def makeListOfZ0(listEverything):
    listZ0 = []
    for item in listEverything:
        listZ0.append(item[1]['Z0'])
    return listZ0

In [10]:
def makeListOfCamHeight(listEverything):
    listCamHeight = []
    for item in listEverything:
        listCamHeight.append(item[1]['cam_height'])
    return listCamHeight

In [11]:
def makeListOfDy(listEverything):
    listDy = []
    for item in listEverything:
        listDy.append(item[1]['dy'])
    return listDy

In [12]:
def makeListOfFocalLength(listEverything):
    listFocalLength = []
    for item in listEverything:
        listFocalLength.append(item[1]['focal_length'])
    return listFocalLength

In [13]:
def makeListOfInvRad(listEverything):
    listInvRad = []
    for item in listEverything:
        listInvRad.append(item[1]['inv_rad'])
    return listInvRad

In [14]:
def makeListOfRoadWidth(listEverything):
    listRoadWidth = []
    for item in listEverything:
        listRoadWidth.append(item[1]['road_width'])
    return listRoadWidth

In [15]:
def makeListOfStraight(listEverything):
    listStraight = []
    for item in listEverything:
        listStraight.append(item[1]['straight'])
    return listStraight

In [16]:
def makeImageList(listEverything):
    imList = []
    for item in listEverything:
        imList.append(item[0])
    return imList

In [17]:
def trainAndTestSplit(listEverything, splitPercent):
    train, test = train_test_split(listEverything, test_size = splitPercent)
    return train, test

In [18]:
def prepPixels(listEverything):
    listNew = []
    current = 0
    for item in listEverything:
        itemNew0 = item[0].astype('float32')
        itemNew0 = item[0]/255
        item_tuple = (itemNew0, item[1])
        current+=1
        #print(current)
        listNew.append(item_tuple)
    return listNew

In [19]:
import time
def runTime(startTime):
    currentTime = time.perf_counter()
    hours = (int)((currentTime - startTime) / 3600)
    minutes = (int)(((currentTime - startTime) - (3600 * hours)) / 60)
    seconds = (int)((currentTime - startTime) - (3600 * hours) - (60 * minutes))
    
    print("\n__Time__", "\nHours:", hours, "\nMinutes:", minutes, "\nSeconds:", seconds)

In [20]:
#only run once per start
list0 = prepPixels(listOfEverything)
 

In [21]:
#only run once per kernel start
train, test = trainAndTestSplit(list0, 0.4)
trainX = makeImageList(train)
trainYforCamHeight = makeListOfCamHeight(train)

crossVal, test = trainAndTestSplit(test, 0.5)
testX = makeImageList(test)
testYforCamHeight = makeListOfCamHeight(test)

crossValX = makeImageList(crossVal)
crossValYforCamHeight = makeListOfCamHeight(crossVal)
import gc
del list0
del train
del test
del crossVal
gc.collect()


22

In [22]:
def modelCreate():
    model =Sequential()

    model.add(Conv2D(3, (5,5), strides=(2,2), input_shape = (480,640,3), activation='elu'))
    
    model.add(Dropout(0.2))


    model.add(Conv2D(24, (5,5), strides=(2,2), activation='elu'))


    model.add(Conv2D(36, (5,5), strides=(2,2), activation='elu'))


    model.add(Conv2D(48, (3,3), strides=(2,2), activation='elu'))


    model.add(Conv2D(64, (3,3), strides=(2,2), activation='elu'))



    model.add(Flatten())

    #next adjustment would be to remove the dropout above and change the activation
    model.add(Dense(1164, activation='elu'))
    model.add(Dense(100, activation='elu'))
    model.add(Dense(50, activation='elu'))
    model.add(Dense(10, activation='elu'))
    
    model.add(Dense(1))
    
    optimizer = Adam(lr=1e-3)
    model.compile(loss='mse', optimizer=optimizer)
        
    print(model)

    return model

In [23]:
startTime = time.perf_counter()
modelOutputDir = os.getcwd()

model = modelCreate()
print('Model has been created')
trainX = numpy.array(trainX)
testX = numpy.array(testX)
crossValX = numpy.array(crossValX)
datagen = ImageDataGenerator(width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=False)
datagen.fit(trainX)
itTrain = datagen.flow(trainX, trainYforCamHeight, batch_size=64)
steps = int(trainX.shape[0] / 64)
checkpointCallback = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(modelOutputDir, 'laneNavModelCamHeight.h5'), verbose=1, save_best_only=True)
print('checkpoint callback done')

history = model.fit_generator(itTrain,
                              steps_per_epoch=steps,
                              epochs=5,
                              validation_data=(crossValX,crossValYforCamHeight),
                              verbose=1,
                              shuffle=1,)
print('history is done')

#model.fit(trainX, trainYforYAW0, epochs=10, batch_size=64, verbose=0)
model.save(os.path.join(modelOutputDir, 'laneNavModelCamHeight.h5'))
print('model has been saved')

runTime(startTime)

Model has been created
checkpoint callback done
Epoch 1/5
45/45 [==============================] - 150s 3s/step - loss: 7.8643 - val_loss: 0.0114
Epoch 2/5
45/45 [==============================] - 145s 3s/step - loss: 0.0183 - val_loss: 0.0107
Epoch 3/5
45/45 [==============================] - 146s 3s/step - loss: 0.0135 - val_loss: 0.0163
Epoch 4/5
45/45 [==============================] - 144s 3s/step - loss: 0.0108 - val_loss: 0.0343
Epoch 5/5
44/45 [============================>.] - ETA: 3s - loss: 0.0099

In [ ]:
pyplot.plot(history.history['loss'], color='blue')
pyplot.plot(history.history['val_loss'], color='red')
pyplot.legend(['training loss', 'validation loss'])

In [26]:

model = load_model('laneNavModelCamHeight.h5')

#def prepComputeImage(imageTest):
    #imageTest = numpy.array(imageTest)
    #imageTest = imageTest.astype('float32')
    #imageTest = imageTest/255
    #params = model.predict(imageTest)
    #return params
#print(trainX.shape)

#testing = testX[5]
#testing = numpy.array(testing)
#testing = testing.reshape(1,480,640,3)

#print(model.predict(testing))
#print(testYforYAW0[5])
current = 0
totalPercentError = 0

for item in testX:
    item = numpy.array(item)
    item = item.reshape(1,480,640,3)
    prediction = model.predict(item)
    print(prediction)
    print(testYforCamHeight[current])
    totalPercentError+=(abs((testYforCamHeight[current]-prediction)/testYforCamHeight[current]))*100
    current+=1
print('done')
print(totalPercentError/960)
print(testX.shape)

[[1.9143997]]
1.8760715386514981
[[1.9354234]]
1.8367196460182411
[[1.9228129]]
1.819353996920784
[[1.9026625]]
1.9356262152715014
[[2.0208287]]
1.8155614285812405
[[2.0402312]]
1.9287495291498749
[[1.9276022]]
1.9250862335154026
[[2.0363352]]
1.8188737785004057
[[1.9174122]]
1.9151718550795267
[[2.0099761]]
1.94920234635966
[[1.9159317]]
1.9276785839777337
[[2.0560308]]
1.843531789904795
[[2.0417178]]
1.9724360752539338
[[2.0385177]]
1.9124871963126653
[[1.932684]]
1.8048633374250171
[[2.0305045]]
1.9546324799445338
[[2.0359752]]
1.9255681020771922
[[2.0311852]]
1.8185289334028196
[[1.9269367]]
1.9239684268307429
[[2.0392823]]
1.9269755032188507
[[2.050359]]
1.8079352256860344
[[1.9389828]]
1.914838392575585
[[1.9197224]]
1.8505887540222992
[[1.9363776]]
1.9764289941809834
[[1.9053658]]
1.824938922560274
[[2.0149367]]
1.895850386220677
[[1.941504]]
1.9753071845076031
[[2.0391586]]
1.9118432987765184
[[1.9430813]]
1.923881155771842
[[1.9073353]]
1.8288600404197477
[[2.023756]]
1.827508